In [63]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_val_score,train_test_split
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeRegressor,DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV, ParameterGrid, StratifiedKFold
from sklearn.linear_model import LinearRegression,LogisticRegression, LassoCV, RidgeCV, ElasticNetCV
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import GridSearchCV, ParameterGrid, StratifiedKFold, RandomizedSearchCV
import itertools as it
import time as time
from catboost import CatBoostRegressor
import xgboost as xgb

In [50]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
nan_cols = test.isna().sum()
test1 = test.drop(nan_cols[nan_cols > 269].index.tolist(), axis = 1)
train1 = train.drop(nan_cols[nan_cols > 269].index.tolist(), axis = 1)

from sklearn.impute import KNNImputer

imputer = KNNImputer(n_neighbors=3)

# fit and transform the data
test_imputed = pd.DataFrame(imputer.fit_transform(test1), columns=test1.columns)
train_imputed = pd.DataFrame(imputer.fit_transform(train1), columns=train1.columns)

corr_matrix = train_imputed.corr().abs()

corr_y = train_imputed.corrwith(train_imputed['y'])
train_clean = train_imputed.drop(corr_y[corr_y.isna()].index.tolist(), axis = 1)
test_clean = test_imputed.drop(corr_y[corr_y.isna()].index.tolist(), axis = 1)

# identify predictors that have correlation of > 0.85
indices = np.where(np.triu(corr_matrix > 0.85, k=1))

# print the pairs of columns that have a correlation > 0.85
high_corr = []
count = 0
for idx in range(len(indices[0])):
    col1 = corr_matrix.columns[indices[0][idx]]
    col2 = corr_matrix.columns[indices[1][idx]]
    high_corr.append(col1)
    count+=1
    #print(f"{col1} and {col2}")
#print(count)

unique_vals = set(high_corr)
train_clean = train_clean.drop(unique_vals, axis = 1)

test_col = train_clean.drop('y', axis = 1)
test_clean = test_clean[test_col.columns]

corr_matrix = train_clean.corr().abs()

corr_y = train_clean.corrwith(train_clean['y'])

low_corr_cols = corr_y[corr_y < 0.01].index.tolist() # 0.01 was best so far 

train_clean = train_clean.drop(low_corr_cols, axis = 1)
test_col = train_clean.drop('y', axis = 1)
test_clean = test_clean[test_col.columns]

X = train_clean.drop('y', axis = 1)
y = train_clean.y

X_test = test_clean

In [22]:
# most important predictors from mars model degree 1
important_mars = ['x102','x561','x355','x591','x488','x430','x670','x572','x581','x014','x118','x755','x756']

In [25]:
set1 = set(train_clean.columns)
set2 = set(important_mars)

set2.difference(set1)

{'x014',
 'x102',
 'x118',
 'x355',
 'x430',
 'x488',
 'x561',
 'x572',
 'x581',
 'x670'}

In [51]:
X['x102'] = train_imputed['x102']
X['x561'] = train_imputed['x561']
X['x355'] = train_imputed['x355']

In [52]:
X_test['x102'] = test_imputed['x102']
X_test['x561'] = test_imputed['x561']
X_test['x355'] = test_imputed['x355']

In [59]:
# important predictors from xgboost random_state=1,max_depth=4,n_estimators=500,learning_rate=0.05,reg_lambda=0.05
important_xgboost = ['x585','x146','x248','x114','x506','x454','x420','x116','x640','x243']

set1 = set(X.columns)
set2 = set(important_xgboost)

set2.difference(set1)

{'x114', 'x116', 'x146', 'x243', 'x420', 'x454', 'x506', 'x585', 'x640'}

In [89]:
xgb_model = xgb.XGBRegressor(random_state=1,max_depth=6,n_estimators=500,learning_rate=0.01,reg_lambda=0.001)
xgb_model.fit(X,y)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.01, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=6, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             n_estimators=500, n_jobs=None, num_parallel_tree=None,
             predictor=None, random_state=1, ...)

In [90]:
catboost = CatBoostRegressor(subsample=0.5,
                          reg_lambda=10, 
                          num_leaves=31, 
                          n_estimators=500, 
                          max_depth=6, 
                          learning_rate=0.05)
catboost.fit(X,y)

0:	learn: 11.8947165	total: 41.5ms	remaining: 20.7s
1:	learn: 11.7858578	total: 59.2ms	remaining: 14.7s
2:	learn: 11.6676041	total: 76.8ms	remaining: 12.7s
3:	learn: 11.5503871	total: 97.1ms	remaining: 12s
4:	learn: 11.4566742	total: 119ms	remaining: 11.8s
5:	learn: 11.3622512	total: 142ms	remaining: 11.7s
6:	learn: 11.2886873	total: 160ms	remaining: 11.3s
7:	learn: 11.2028435	total: 176ms	remaining: 10.8s
8:	learn: 11.1191126	total: 185ms	remaining: 10.1s
9:	learn: 11.0411953	total: 194ms	remaining: 9.49s
10:	learn: 10.9725414	total: 201ms	remaining: 8.95s
11:	learn: 10.9052244	total: 209ms	remaining: 8.5s
12:	learn: 10.8305276	total: 218ms	remaining: 8.15s
13:	learn: 10.7653204	total: 229ms	remaining: 7.96s
14:	learn: 10.7042936	total: 238ms	remaining: 7.71s
15:	learn: 10.6388351	total: 265ms	remaining: 8.01s
16:	learn: 10.5773727	total: 296ms	remaining: 8.41s
17:	learn: 10.5256253	total: 310ms	remaining: 8.3s
18:	learn: 10.4797320	total: 325ms	remaining: 8.23s
19:	learn: 10.4287823	

169:	learn: 8.3996648	total: 2.62s	remaining: 5.08s
170:	learn: 8.3915884	total: 2.64s	remaining: 5.08s
171:	learn: 8.3875297	total: 2.65s	remaining: 5.06s
172:	learn: 8.3858500	total: 2.66s	remaining: 5.03s
173:	learn: 8.3834394	total: 2.67s	remaining: 5.01s
174:	learn: 8.3751527	total: 2.69s	remaining: 5s
175:	learn: 8.3729358	total: 2.7s	remaining: 4.98s
176:	learn: 8.3664345	total: 2.72s	remaining: 4.96s
177:	learn: 8.3609674	total: 2.74s	remaining: 4.96s
178:	learn: 8.3560123	total: 2.75s	remaining: 4.94s
179:	learn: 8.3518582	total: 2.76s	remaining: 4.91s
180:	learn: 8.3488994	total: 2.78s	remaining: 4.89s
181:	learn: 8.3485062	total: 2.79s	remaining: 4.88s
182:	learn: 8.3410373	total: 2.8s	remaining: 4.85s
183:	learn: 8.3361577	total: 2.81s	remaining: 4.82s
184:	learn: 8.3210731	total: 2.82s	remaining: 4.8s
185:	learn: 8.3184240	total: 2.84s	remaining: 4.8s
186:	learn: 8.3164465	total: 2.85s	remaining: 4.77s
187:	learn: 8.3158143	total: 2.86s	remaining: 4.75s
188:	learn: 8.30962

339:	learn: 7.5241695	total: 4.24s	remaining: 2s
340:	learn: 7.5183186	total: 4.25s	remaining: 1.98s
341:	learn: 7.5163228	total: 4.26s	remaining: 1.97s
342:	learn: 7.5075665	total: 4.27s	remaining: 1.96s
343:	learn: 7.5072907	total: 4.28s	remaining: 1.94s
344:	learn: 7.5019824	total: 4.29s	remaining: 1.93s
345:	learn: 7.4954952	total: 4.29s	remaining: 1.91s
346:	learn: 7.4834217	total: 4.3s	remaining: 1.9s
347:	learn: 7.4827744	total: 4.31s	remaining: 1.88s
348:	learn: 7.4815990	total: 4.31s	remaining: 1.87s
349:	learn: 7.4776786	total: 4.32s	remaining: 1.85s
350:	learn: 7.4751307	total: 4.33s	remaining: 1.84s
351:	learn: 7.4705743	total: 4.33s	remaining: 1.82s
352:	learn: 7.4632299	total: 4.34s	remaining: 1.81s
353:	learn: 7.4528653	total: 4.35s	remaining: 1.79s
354:	learn: 7.4492537	total: 4.36s	remaining: 1.78s
355:	learn: 7.4470789	total: 4.36s	remaining: 1.76s
356:	learn: 7.4375709	total: 4.37s	remaining: 1.75s
357:	learn: 7.4323551	total: 4.38s	remaining: 1.74s
358:	learn: 7.426

In [91]:
#Finding the lasso coefficients for increasing values of the tuning parameter
from sklearn.linear_model import Ridge, RidgeCV, Lasso, LassoCV

alphas = 10**np.linspace(10,-2,100)*0.1
lasso = Lasso(max_iter = 10000)
coefs = []

for a in alphas:
    lasso.set_params(alpha=a)
    lasso.fit(X,y)
    coefs.append(lasso.coef_)

alphas = 10**np.linspace(-1,-5,200)*0.5
lassocv = LassoCV(alphas = alphas, cv = 10, max_iter = 100000)
lassocv.fit(X,y)

#Optimal value of the tuning parameter - lamda
lassocv.alpha_

0.007851450623646887

In [92]:
lasso = Lasso(alpha = lassocv.alpha_)
lasso.fit(X,y)

Lasso(alpha=0.007851450623646887)

In [93]:
en = StackingRegressor(estimators = [('lasso', lasso),('catboost', catboost),('xgb', xgb_model)],
                     final_estimator=LinearRegression(),                                          
                    cv = KFold(n_splits = 5, shuffle = True, random_state=1))
en.fit(X,y)

0:	learn: 11.8947165	total: 145ms	remaining: 1m 12s
1:	learn: 11.7858578	total: 319ms	remaining: 1m 19s
2:	learn: 11.6676041	total: 342ms	remaining: 56.7s
3:	learn: 11.5503871	total: 391ms	remaining: 48.5s
4:	learn: 11.4566742	total: 439ms	remaining: 43.5s
5:	learn: 11.3622512	total: 482ms	remaining: 39.7s
6:	learn: 11.2886873	total: 520ms	remaining: 36.6s
7:	learn: 11.2028435	total: 532ms	remaining: 32.7s
8:	learn: 11.1191126	total: 543ms	remaining: 29.6s
9:	learn: 11.0411953	total: 562ms	remaining: 27.5s
10:	learn: 10.9725414	total: 573ms	remaining: 25.5s
11:	learn: 10.9052244	total: 583ms	remaining: 23.7s
12:	learn: 10.8305276	total: 594ms	remaining: 22.3s
13:	learn: 10.7653204	total: 619ms	remaining: 21.5s
14:	learn: 10.7042936	total: 633ms	remaining: 20.5s
15:	learn: 10.6388351	total: 644ms	remaining: 19.5s
16:	learn: 10.5773727	total: 655ms	remaining: 18.6s
17:	learn: 10.5256253	total: 666ms	remaining: 17.8s
18:	learn: 10.4797320	total: 677ms	remaining: 17.1s
19:	learn: 10.428782

166:	learn: 8.4124331	total: 2.23s	remaining: 4.44s
167:	learn: 8.4104312	total: 2.24s	remaining: 4.42s
168:	learn: 8.4019231	total: 2.25s	remaining: 4.4s
169:	learn: 8.3996648	total: 2.26s	remaining: 4.38s
170:	learn: 8.3915884	total: 2.26s	remaining: 4.36s
171:	learn: 8.3875297	total: 2.27s	remaining: 4.33s
172:	learn: 8.3858500	total: 2.28s	remaining: 4.31s
173:	learn: 8.3834394	total: 2.29s	remaining: 4.29s
174:	learn: 8.3751527	total: 2.3s	remaining: 4.27s
175:	learn: 8.3729358	total: 2.31s	remaining: 4.25s
176:	learn: 8.3664345	total: 2.32s	remaining: 4.23s
177:	learn: 8.3609674	total: 2.33s	remaining: 4.21s
178:	learn: 8.3560123	total: 2.34s	remaining: 4.19s
179:	learn: 8.3518582	total: 2.34s	remaining: 4.17s
180:	learn: 8.3488994	total: 2.35s	remaining: 4.14s
181:	learn: 8.3485062	total: 2.36s	remaining: 4.13s
182:	learn: 8.3410373	total: 2.38s	remaining: 4.12s
183:	learn: 8.3361577	total: 2.39s	remaining: 4.1s
184:	learn: 8.3210731	total: 2.4s	remaining: 4.08s
185:	learn: 8.31

338:	learn: 7.5262494	total: 3.81s	remaining: 1.81s
339:	learn: 7.5241695	total: 3.82s	remaining: 1.8s
340:	learn: 7.5183186	total: 3.83s	remaining: 1.79s
341:	learn: 7.5163228	total: 3.84s	remaining: 1.77s
342:	learn: 7.5075665	total: 3.85s	remaining: 1.76s
343:	learn: 7.5072907	total: 3.85s	remaining: 1.75s
344:	learn: 7.5019824	total: 3.86s	remaining: 1.74s
345:	learn: 7.4954952	total: 3.87s	remaining: 1.72s
346:	learn: 7.4834217	total: 3.88s	remaining: 1.71s
347:	learn: 7.4827744	total: 3.88s	remaining: 1.7s
348:	learn: 7.4815990	total: 3.89s	remaining: 1.68s
349:	learn: 7.4776786	total: 3.9s	remaining: 1.67s
350:	learn: 7.4751307	total: 3.91s	remaining: 1.66s
351:	learn: 7.4705743	total: 3.91s	remaining: 1.65s
352:	learn: 7.4632299	total: 3.92s	remaining: 1.63s
353:	learn: 7.4528653	total: 3.93s	remaining: 1.62s
354:	learn: 7.4492537	total: 3.93s	remaining: 1.61s
355:	learn: 7.4470789	total: 3.94s	remaining: 1.59s
356:	learn: 7.4375709	total: 3.95s	remaining: 1.58s
357:	learn: 7.4

0:	learn: 11.6057671	total: 32.1ms	remaining: 16s
1:	learn: 11.5102633	total: 46.6ms	remaining: 11.6s
2:	learn: 11.4061109	total: 60.2ms	remaining: 9.97s
3:	learn: 11.3103852	total: 72.3ms	remaining: 8.96s
4:	learn: 11.2124662	total: 81.9ms	remaining: 8.11s
5:	learn: 11.1410411	total: 91.2ms	remaining: 7.51s
6:	learn: 11.0638301	total: 101ms	remaining: 7.13s
7:	learn: 10.9843436	total: 111ms	remaining: 6.84s
8:	learn: 10.8957310	total: 121ms	remaining: 6.61s
9:	learn: 10.8347100	total: 131ms	remaining: 6.44s
10:	learn: 10.7655566	total: 141ms	remaining: 6.29s
11:	learn: 10.6927448	total: 151ms	remaining: 6.15s
12:	learn: 10.6179538	total: 161ms	remaining: 6.03s
13:	learn: 10.5629115	total: 174ms	remaining: 6.03s
14:	learn: 10.4924649	total: 185ms	remaining: 5.98s
15:	learn: 10.4301720	total: 194ms	remaining: 5.88s
16:	learn: 10.3735596	total: 205ms	remaining: 5.82s
17:	learn: 10.3339657	total: 214ms	remaining: 5.74s
18:	learn: 10.2921055	total: 224ms	remaining: 5.66s
19:	learn: 10.2465

166:	learn: 8.2067761	total: 2.1s	remaining: 4.18s
167:	learn: 8.2055893	total: 2.11s	remaining: 4.18s
168:	learn: 8.1980309	total: 2.12s	remaining: 4.16s
169:	learn: 8.1911989	total: 2.18s	remaining: 4.24s
170:	learn: 8.1856268	total: 2.21s	remaining: 4.25s
171:	learn: 8.1745111	total: 2.24s	remaining: 4.27s
172:	learn: 8.1637781	total: 2.28s	remaining: 4.31s
173:	learn: 8.1572748	total: 2.3s	remaining: 4.3s
174:	learn: 8.1563102	total: 2.32s	remaining: 4.3s
175:	learn: 8.1492254	total: 2.36s	remaining: 4.34s
176:	learn: 8.1437839	total: 2.37s	remaining: 4.32s
177:	learn: 8.1349325	total: 2.37s	remaining: 4.29s
178:	learn: 8.1314750	total: 2.38s	remaining: 4.27s
179:	learn: 8.1307425	total: 2.39s	remaining: 4.25s
180:	learn: 8.1265375	total: 2.4s	remaining: 4.22s
181:	learn: 8.1247092	total: 2.4s	remaining: 4.2s
182:	learn: 8.1201676	total: 2.41s	remaining: 4.18s
183:	learn: 8.1180680	total: 2.42s	remaining: 4.16s
184:	learn: 8.1171422	total: 2.43s	remaining: 4.14s
185:	learn: 8.10683

...

360:	learn: 7.4107957	total: 4.36s	remaining: 1.68s
361:	learn: 7.4031709	total: 4.38s	remaining: 1.67s
362:	learn: 7.3997735	total: 4.4s	remaining: 1.66s
363:	learn: 7.3981709	total: 4.41s	remaining: 1.65s
364:	learn: 7.3884421	total: 4.42s	remaining: 1.64s
365:	learn: 7.3839048	total: 4.44s	remaining: 1.62s
366:	learn: 7.3829996	total: 4.45s	remaining: 1.61s
367:	learn: 7.3740205	total: 4.46s	remaining: 1.6s
368:	learn: 7.3730043	total: 4.47s	remaining: 1.59s
369:	learn: 7.3620757	total: 4.48s	remaining: 1.57s
370:	learn: 7.3571602	total: 4.5s	remaining: 1.56s
371:	learn: 7.3517109	total: 4.51s	remaining: 1.55s
372:	learn: 7.3441296	total: 4.52s	remaining: 1.54s
373:	learn: 7.3435949	total: 4.53s	remaining: 1.52s
374:	learn: 7.3415041	total: 4.55s	remaining: 1.51s
375:	learn: 7.3370102	total: 4.57s	remaining: 1.5s
376:	learn: 7.3278803	total: 4.59s	remaining: 1.5s
377:	learn: 7.3220070	total: 4.59s	remaining: 1.48s
378:	learn: 7.3191197	total: 4.61s	remaining: 1.47s
379:	learn: 7.317

24:	learn: 10.1205524	total: 296ms	remaining: 5.62s
25:	learn: 10.0883350	total: 318ms	remaining: 5.8s
26:	learn: 10.0612953	total: 331ms	remaining: 5.79s
27:	learn: 10.0211451	total: 343ms	remaining: 5.78s
28:	learn: 9.9920777	total: 353ms	remaining: 5.73s
29:	learn: 9.9563979	total: 363ms	remaining: 5.68s
30:	learn: 9.9212060	total: 373ms	remaining: 5.64s
31:	learn: 9.8839847	total: 385ms	remaining: 5.62s
32:	learn: 9.8501259	total: 395ms	remaining: 5.59s
33:	learn: 9.8086055	total: 406ms	remaining: 5.56s
34:	learn: 9.7923912	total: 417ms	remaining: 5.54s
35:	learn: 9.7526722	total: 427ms	remaining: 5.51s
36:	learn: 9.7277269	total: 437ms	remaining: 5.47s
37:	learn: 9.6982433	total: 448ms	remaining: 5.44s
38:	learn: 9.6779344	total: 459ms	remaining: 5.42s
39:	learn: 9.6534262	total: 484ms	remaining: 5.56s
40:	learn: 9.6250603	total: 495ms	remaining: 5.55s
41:	learn: 9.6021575	total: 513ms	remaining: 5.6s
42:	learn: 9.5744316	total: 536ms	remaining: 5.69s
43:	learn: 9.5635710	total: 5

192:	learn: 8.1331770	total: 1.91s	remaining: 3.04s
193:	learn: 8.1263764	total: 1.93s	remaining: 3.04s
194:	learn: 8.1205869	total: 1.95s	remaining: 3.04s
195:	learn: 8.1071980	total: 1.95s	remaining: 3.03s
196:	learn: 8.0984303	total: 1.96s	remaining: 3.02s
197:	learn: 8.0816952	total: 1.97s	remaining: 3s
198:	learn: 8.0793998	total: 1.98s	remaining: 2.99s
199:	learn: 8.0768048	total: 1.99s	remaining: 2.98s
200:	learn: 8.0699900	total: 1.99s	remaining: 2.97s
201:	learn: 8.0590324	total: 2s	remaining: 2.95s
202:	learn: 8.0547596	total: 2.01s	remaining: 2.94s
203:	learn: 8.0482560	total: 2.02s	remaining: 2.93s
204:	learn: 8.0437452	total: 2.03s	remaining: 2.92s
205:	learn: 8.0285741	total: 2.04s	remaining: 2.9s
206:	learn: 8.0249885	total: 2.04s	remaining: 2.89s
207:	learn: 8.0222361	total: 2.05s	remaining: 2.88s
208:	learn: 8.0181579	total: 2.06s	remaining: 2.87s
209:	learn: 8.0149191	total: 2.07s	remaining: 2.85s
210:	learn: 8.0123565	total: 2.08s	remaining: 2.84s
211:	learn: 8.00854

356:	learn: 7.1873970	total: 3.34s	remaining: 1.34s
357:	learn: 7.1861419	total: 3.35s	remaining: 1.33s
358:	learn: 7.1851638	total: 3.36s	remaining: 1.32s
359:	learn: 7.1829570	total: 3.37s	remaining: 1.31s
360:	learn: 7.1783811	total: 3.38s	remaining: 1.3s
361:	learn: 7.1739218	total: 3.39s	remaining: 1.29s
362:	learn: 7.1695703	total: 3.4s	remaining: 1.28s
363:	learn: 7.1679554	total: 3.4s	remaining: 1.27s
364:	learn: 7.1552072	total: 3.41s	remaining: 1.26s
365:	learn: 7.1475913	total: 3.42s	remaining: 1.25s
366:	learn: 7.1462190	total: 3.43s	remaining: 1.24s
367:	learn: 7.1329149	total: 3.43s	remaining: 1.23s
368:	learn: 7.1289052	total: 3.44s	remaining: 1.22s
369:	learn: 7.1277317	total: 3.45s	remaining: 1.21s
370:	learn: 7.1222374	total: 3.46s	remaining: 1.2s
371:	learn: 7.1126593	total: 3.46s	remaining: 1.19s
372:	learn: 7.1102933	total: 3.47s	remaining: 1.18s
373:	learn: 7.1082772	total: 3.48s	remaining: 1.17s
374:	learn: 7.1061001	total: 3.49s	remaining: 1.16s
375:	learn: 7.10

StackingRegressor(cv=KFold(n_splits=5, random_state=1, shuffle=True),
                  estimators=[('lasso', Lasso(alpha=0.007851450623646887)),
                              ('catboost',
                               <catboost.core.CatBoostRegressor object at 0x7f8983d59820>),
                              ('xgb',
                               XGBRegressor(base_score=None, booster=None,
                                            callbacks=None,
                                            colsample_bylevel=None,
                                            colsample_bynode=None,
                                            colsample_bytree=None,
                                            early_stopping_rounds=None,...
                                            importance_type=None,
                                            interaction_constraints=None,
                                            learning_rate=0.01, max_bin=None,
                                            max_cat_threshold=None,
                                            max_cat_to_onehot=None,
                                            max_delta_step=None, max_depth=6,
                                            max_leaves=None,
                                            min_child_weight=None, missing=nan,
                                            monotone_constraints=None,
                                            n_estimators=500, n_jobs=None,
                                            num_parallel_tree=None,
                                            predictor=None, random_state=1, ...))],
                  final_estimator=LinearRegression())

In [94]:
pred = pd.DataFrame()
pred['id'] = test['id']
pred['y'] = en.predict(X_test)
pred = pred.set_index(['id'])
pred.to_csv('Witarsa_Ashley_regression.csv') # RMSE: 9.39!!!!! w only CatBoost & Lasso, 9.44 + XGBoost

### With scaling

In [ ]:
scaler = StandardScaler().fit(X)
X_scaled = scaler.transform(X)
X_test_scaled = scaler.transform(X_test)

from sklearn.decomposition import PCA
plt.rcParams.update(plt.rcParamsDefault)
pca = PCA()
pca.fit(X_scaled)

pca = PCA(n_components = 0.99)
pca.fit(X_scaled)

X_train_pca = pca.transform(X_scaled)
X_test_pca = pca.transform(X_test_scaled)

In [9]:
#K-fold cross validation to find optimal parameters for CatBoost regressor
start_time = time.time()
param_grid = {'max_depth': [4,6,8],
              'num_leaves': [20, 31, 40],
              'learning_rate': [0.01, 0.05, 0.1],
               'reg_lambda':[0, 10, 100],
                'n_estimators':[100, 500, 1000],
                'subsample': [0.5, 0.75, 1.0]}

cv = KFold(n_splits=5,shuffle=True,random_state=1)
optimal_params = RandomizedSearchCV(estimator=CatBoostRegressor(random_state=1, verbose=False),                                                       
                             param_distributions = param_grid, n_iter = 200,
                             verbose = 1,random_state = 1,
                             n_jobs=-1,
                             cv = cv)
optimal_params.fit(X_train_pca,y)
print("Optimal parameter values =", optimal_params.best_params_)
print("Optimal cross validation R-squared = ",optimal_params.best_score_)
print("Time taken = ", round((time.time()-start_time)/60), " minutes")

Fitting 5 folds for each of 200 candidates, totalling 1000 fits


/Users/ashleywitarsa/opt/anaconda3/lib/python3.8/site-packages/sklearn/model_selection/_validation.py:378: FitFailedWarning: 
645 fits failed out of a total of 1000.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
645 fits failed with the following error:
Traceback (most recent call last):
  File "/Users/ashleywitarsa/opt/anaconda3/lib/python3.8/site-packages/sklearn/model_selection/_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/Users/ashleywitarsa/opt/anaconda3/lib/python3.8/site-packages/catboost/core.py", line 5730, in fit
    return self._fit(X, y, cat_features, text_features, embedding_features, None, sample_weight, None, None, None, None, baseline,
  File "/Users/ashleywitars

Optimal parameter values = {'subsample': 0.5, 'reg_lambda': 10, 'num_leaves': 31, 'n_estimators': 500, 'max_depth': 6, 'learning_rate': 0.05}
Optimal cross validation R-squared =  0.27092195420662074
Time taken =  57  minutes


In [15]:
catboost = CatBoostRegressor(subsample=0.5,
                          reg_lambda=10, 
                          num_leaves=31, 
                          n_estimators=500, 
                          max_depth=6, 
                          learning_rate=0.05)
catboost.fit(X_train_pca,y)

0:	learn: 11.9344848	total: 21.1ms	remaining: 10.5s
1:	learn: 11.8732524	total: 30.5ms	remaining: 7.58s
2:	learn: 11.8083256	total: 43.3ms	remaining: 7.18s
3:	learn: 11.7441644	total: 56.7ms	remaining: 7.03s
4:	learn: 11.6747699	total: 68.3ms	remaining: 6.76s
5:	learn: 11.6012707	total: 77.1ms	remaining: 6.35s
6:	learn: 11.5300387	total: 88.1ms	remaining: 6.2s
7:	learn: 11.4722839	total: 102ms	remaining: 6.29s
8:	learn: 11.4014505	total: 115ms	remaining: 6.3s
9:	learn: 11.3388452	total: 130ms	remaining: 6.36s
10:	learn: 11.2734005	total: 154ms	remaining: 6.83s
11:	learn: 11.2086888	total: 173ms	remaining: 7.03s
12:	learn: 11.1631937	total: 185ms	remaining: 6.94s
13:	learn: 11.1276145	total: 200ms	remaining: 6.96s
14:	learn: 11.0774615	total: 215ms	remaining: 6.95s
15:	learn: 11.0312453	total: 235ms	remaining: 7.12s
16:	learn: 10.9810156	total: 260ms	remaining: 7.39s
17:	learn: 10.9302715	total: 274ms	remaining: 7.33s
18:	learn: 10.8902815	total: 285ms	remaining: 7.22s
19:	learn: 10.850

164:	learn: 8.6271040	total: 3.52s	remaining: 7.15s
165:	learn: 8.6218445	total: 3.55s	remaining: 7.14s
166:	learn: 8.6154784	total: 3.56s	remaining: 7.1s
167:	learn: 8.6082696	total: 3.58s	remaining: 7.07s
168:	learn: 8.6026565	total: 3.59s	remaining: 7.03s
169:	learn: 8.5965337	total: 3.6s	remaining: 6.99s
170:	learn: 8.5876294	total: 3.62s	remaining: 6.97s
171:	learn: 8.5734012	total: 3.63s	remaining: 6.93s
172:	learn: 8.5587942	total: 3.64s	remaining: 6.88s
173:	learn: 8.5548828	total: 3.65s	remaining: 6.84s
174:	learn: 8.5479404	total: 3.67s	remaining: 6.81s
175:	learn: 8.5414653	total: 3.68s	remaining: 6.77s
176:	learn: 8.5363731	total: 3.69s	remaining: 6.73s
177:	learn: 8.5285333	total: 3.7s	remaining: 6.69s
178:	learn: 8.5231049	total: 3.71s	remaining: 6.66s
179:	learn: 8.5175682	total: 3.73s	remaining: 6.63s
180:	learn: 8.5144832	total: 3.74s	remaining: 6.59s
181:	learn: 8.5061215	total: 3.75s	remaining: 6.56s
182:	learn: 8.5040077	total: 3.77s	remaining: 6.53s
183:	learn: 8.4

340:	learn: 7.4579058	total: 7.07s	remaining: 3.29s
341:	learn: 7.4543178	total: 7.08s	remaining: 3.27s
342:	learn: 7.4474686	total: 7.09s	remaining: 3.24s
343:	learn: 7.4445805	total: 7.11s	remaining: 3.22s
344:	learn: 7.4387850	total: 7.12s	remaining: 3.2s
345:	learn: 7.4351469	total: 7.13s	remaining: 3.17s
346:	learn: 7.4279133	total: 7.16s	remaining: 3.16s
347:	learn: 7.4269923	total: 7.17s	remaining: 3.13s
348:	learn: 7.4236062	total: 7.18s	remaining: 3.11s
349:	learn: 7.4211318	total: 7.19s	remaining: 3.08s
350:	learn: 7.4190650	total: 7.21s	remaining: 3.06s
351:	learn: 7.4144476	total: 7.21s	remaining: 3.03s
352:	learn: 7.4097334	total: 7.22s	remaining: 3.01s
353:	learn: 7.4020234	total: 7.23s	remaining: 2.98s
354:	learn: 7.3967075	total: 7.24s	remaining: 2.96s
355:	learn: 7.3857634	total: 7.25s	remaining: 2.93s
356:	learn: 7.3804616	total: 7.27s	remaining: 2.91s
357:	learn: 7.3741248	total: 7.28s	remaining: 2.89s
358:	learn: 7.3703107	total: 7.29s	remaining: 2.86s
359:	learn: 7

In [14]:
pred = pd.DataFrame()
pred['id'] = test['id']
pred['y'] = model.predict(X_test_pca)
pred = pred.set_index(['id'])
pred.to_csv('Witarsa_Ashley_regression.csv') # RMSE: 9.717

#### Stack lasso and CatBoost

In [18]:
from sklearn.linear_model import Ridge, RidgeCV, Lasso, LassoCV

lasso = Lasso(alpha = 0.05)
lasso.fit(X_train_pca, y)

Lasso(alpha=0.05)

In [19]:
from sklearn.ensemble import StackingRegressor

en = StackingRegressor(estimators = [('lasso', lasso),('catboost', catboost)],
                     final_estimator=LinearRegression(),                                          
                    cv = KFold(n_splits = 5, shuffle = True, random_state=1))
en.fit(X_train_pca,y)


0:	learn: 11.9344848	total: 18.3ms	remaining: 9.11s
1:	learn: 11.8732524	total: 28.3ms	remaining: 7.05s
2:	learn: 11.8083256	total: 38.1ms	remaining: 6.31s
3:	learn: 11.7441644	total: 45.5ms	remaining: 5.64s
4:	learn: 11.6747699	total: 55.9ms	remaining: 5.53s
5:	learn: 11.6012707	total: 79.5ms	remaining: 6.55s
6:	learn: 11.5300387	total: 86.9ms	remaining: 6.12s
7:	learn: 11.4722839	total: 94.6ms	remaining: 5.82s
8:	learn: 11.4014505	total: 102ms	remaining: 5.58s
9:	learn: 11.3388452	total: 125ms	remaining: 6.13s
10:	learn: 11.2734005	total: 133ms	remaining: 5.9s
11:	learn: 11.2086888	total: 145ms	remaining: 5.91s
12:	learn: 11.1631937	total: 153ms	remaining: 5.71s
13:	learn: 11.1276145	total: 171ms	remaining: 5.93s
14:	learn: 11.0774615	total: 180ms	remaining: 5.83s
15:	learn: 11.0312453	total: 188ms	remaining: 5.68s
16:	learn: 10.9810156	total: 195ms	remaining: 5.54s
17:	learn: 10.9302715	total: 203ms	remaining: 5.42s
18:	learn: 10.8902815	total: 210ms	remaining: 5.31s
19:	learn: 10.8

165:	learn: 8.6218445	total: 3.2s	remaining: 6.44s
166:	learn: 8.6154784	total: 3.23s	remaining: 6.44s
167:	learn: 8.6082696	total: 3.24s	remaining: 6.4s
168:	learn: 8.6026565	total: 3.25s	remaining: 6.38s
169:	learn: 8.5965337	total: 3.28s	remaining: 6.36s
170:	learn: 8.5876294	total: 3.29s	remaining: 6.33s
171:	learn: 8.5734012	total: 3.3s	remaining: 6.3s
172:	learn: 8.5587942	total: 3.32s	remaining: 6.27s
173:	learn: 8.5548828	total: 3.33s	remaining: 6.23s
174:	learn: 8.5479404	total: 3.33s	remaining: 6.19s
175:	learn: 8.5414653	total: 3.34s	remaining: 6.16s
176:	learn: 8.5363731	total: 3.36s	remaining: 6.12s
177:	learn: 8.5285333	total: 3.37s	remaining: 6.1s
178:	learn: 8.5231049	total: 3.38s	remaining: 6.06s
179:	learn: 8.5175682	total: 3.39s	remaining: 6.02s
180:	learn: 8.5144832	total: 3.4s	remaining: 6s
181:	learn: 8.5061215	total: 3.42s	remaining: 5.98s
182:	learn: 8.5040077	total: 3.43s	remaining: 5.95s
183:	learn: 8.4978141	total: 3.44s	remaining: 5.91s
184:	learn: 8.4908503

333:	learn: 7.4974612	total: 5.77s	remaining: 2.87s
334:	learn: 7.4960495	total: 5.79s	remaining: 2.85s
335:	learn: 7.4838617	total: 5.8s	remaining: 2.83s
336:	learn: 7.4800373	total: 5.82s	remaining: 2.81s
337:	learn: 7.4746849	total: 5.83s	remaining: 2.79s
338:	learn: 7.4681739	total: 5.85s	remaining: 2.78s
339:	learn: 7.4604793	total: 5.87s	remaining: 2.76s
340:	learn: 7.4579058	total: 5.88s	remaining: 2.74s
341:	learn: 7.4543178	total: 5.92s	remaining: 2.73s
342:	learn: 7.4474686	total: 5.93s	remaining: 2.71s
343:	learn: 7.4445805	total: 5.94s	remaining: 2.69s
344:	learn: 7.4387850	total: 5.97s	remaining: 2.68s
345:	learn: 7.4351469	total: 5.99s	remaining: 2.66s
346:	learn: 7.4279133	total: 6.01s	remaining: 2.65s
347:	learn: 7.4269923	total: 6.02s	remaining: 2.63s
348:	learn: 7.4236062	total: 6.03s	remaining: 2.61s
349:	learn: 7.4211318	total: 6.04s	remaining: 2.59s
350:	learn: 7.4190650	total: 6.06s	remaining: 2.57s
351:	learn: 7.4144476	total: 6.07s	remaining: 2.55s
352:	learn: 7

0:	learn: 11.6377236	total: 17.3ms	remaining: 8.64s
1:	learn: 11.5771998	total: 29.5ms	remaining: 7.34s
2:	learn: 11.5013632	total: 41ms	remaining: 6.8s
3:	learn: 11.4445360	total: 64.1ms	remaining: 7.95s
4:	learn: 11.3642358	total: 75.4ms	remaining: 7.46s
5:	learn: 11.3130743	total: 95.9ms	remaining: 7.9s
6:	learn: 11.2541841	total: 130ms	remaining: 9.14s
7:	learn: 11.1931051	total: 164ms	remaining: 10.1s
8:	learn: 11.1355028	total: 178ms	remaining: 9.73s
9:	learn: 11.0718947	total: 199ms	remaining: 9.76s
10:	learn: 11.0200474	total: 218ms	remaining: 9.7s
11:	learn: 10.9608484	total: 237ms	remaining: 9.63s
12:	learn: 10.9173719	total: 265ms	remaining: 9.93s
13:	learn: 10.8663532	total: 277ms	remaining: 9.63s
14:	learn: 10.8075900	total: 291ms	remaining: 9.41s
15:	learn: 10.7691298	total: 313ms	remaining: 9.48s
16:	learn: 10.7190966	total: 325ms	remaining: 9.24s
17:	learn: 10.6780907	total: 336ms	remaining: 9.01s
18:	learn: 10.6362513	total: 361ms	remaining: 9.14s
19:	learn: 10.5893572

171:	learn: 8.4414011	total: 2.93s	remaining: 5.58s
172:	learn: 8.4285524	total: 2.94s	remaining: 5.55s
173:	learn: 8.4224653	total: 2.97s	remaining: 5.56s
174:	learn: 8.4085549	total: 2.98s	remaining: 5.53s
175:	learn: 8.4060211	total: 2.99s	remaining: 5.5s
176:	learn: 8.3992243	total: 3.01s	remaining: 5.5s
177:	learn: 8.3886143	total: 3.02s	remaining: 5.47s
178:	learn: 8.3841990	total: 3.05s	remaining: 5.46s
179:	learn: 8.3814934	total: 3.06s	remaining: 5.43s
180:	learn: 8.3703369	total: 3.07s	remaining: 5.41s
181:	learn: 8.3670753	total: 3.08s	remaining: 5.38s
182:	learn: 8.3583206	total: 3.1s	remaining: 5.37s
183:	learn: 8.3547321	total: 3.11s	remaining: 5.34s
184:	learn: 8.3518338	total: 3.12s	remaining: 5.32s
185:	learn: 8.3388586	total: 3.15s	remaining: 5.31s
186:	learn: 8.3357940	total: 3.16s	remaining: 5.29s
187:	learn: 8.3240503	total: 3.18s	remaining: 5.28s
188:	learn: 8.3197318	total: 3.2s	remaining: 5.26s
189:	learn: 8.3151672	total: 3.21s	remaining: 5.24s
190:	learn: 8.30

...

336:	learn: 7.3162962	total: 4.43s	remaining: 2.14s
337:	learn: 7.3086360	total: 4.45s	remaining: 2.13s
338:	learn: 7.3066649	total: 4.46s	remaining: 2.12s
339:	learn: 7.3017491	total: 4.47s	remaining: 2.1s
340:	learn: 7.2978177	total: 4.48s	remaining: 2.09s
341:	learn: 7.2931166	total: 4.49s	remaining: 2.07s
342:	learn: 7.2842861	total: 4.51s	remaining: 2.06s
343:	learn: 7.2792572	total: 4.51s	remaining: 2.05s
344:	learn: 7.2708513	total: 4.53s	remaining: 2.03s
345:	learn: 7.2663014	total: 4.54s	remaining: 2.02s
346:	learn: 7.2634364	total: 4.55s	remaining: 2.01s
347:	learn: 7.2539753	total: 4.56s	remaining: 1.99s
348:	learn: 7.2479433	total: 4.57s	remaining: 1.98s
349:	learn: 7.2430956	total: 4.58s	remaining: 1.96s
350:	learn: 7.2373621	total: 4.59s	remaining: 1.95s
351:	learn: 7.2322636	total: 4.61s	remaining: 1.94s
352:	learn: 7.2311830	total: 4.62s	remaining: 1.92s
353:	learn: 7.2220119	total: 4.63s	remaining: 1.91s
354:	learn: 7.2153591	total: 4.64s	remaining: 1.9s
355:	learn: 7.

0:	learn: 11.8000342	total: 22.6ms	remaining: 11.3s
1:	learn: 11.7448350	total: 32.4ms	remaining: 8.07s
2:	learn: 11.6734132	total: 42.6ms	remaining: 7.06s
3:	learn: 11.6115005	total: 63.7ms	remaining: 7.9s
4:	learn: 11.5515560	total: 76.3ms	remaining: 7.56s
5:	learn: 11.4788221	total: 88.4ms	remaining: 7.28s
6:	learn: 11.4226400	total: 101ms	remaining: 7.13s
7:	learn: 11.3585781	total: 123ms	remaining: 7.59s
8:	learn: 11.3017590	total: 133ms	remaining: 7.26s
9:	learn: 11.2480686	total: 143ms	remaining: 7.02s
10:	learn: 11.1893481	total: 156ms	remaining: 6.94s
11:	learn: 11.1296315	total: 174ms	remaining: 7.07s
12:	learn: 11.0822809	total: 187ms	remaining: 6.99s
13:	learn: 11.0347623	total: 198ms	remaining: 6.88s
14:	learn: 10.9771773	total: 222ms	remaining: 7.17s
15:	learn: 10.9336735	total: 244ms	remaining: 7.38s
16:	learn: 10.8841800	total: 269ms	remaining: 7.63s
17:	learn: 10.8476298	total: 283ms	remaining: 7.59s
18:	learn: 10.8017993	total: 296ms	remaining: 7.5s
19:	learn: 10.7704

170:	learn: 8.4714977	total: 2.74s	remaining: 5.27s
171:	learn: 8.4599737	total: 2.77s	remaining: 5.28s
172:	learn: 8.4501346	total: 2.79s	remaining: 5.27s
173:	learn: 8.4471681	total: 2.8s	remaining: 5.25s
174:	learn: 8.4387019	total: 2.81s	remaining: 5.22s
175:	learn: 8.4277289	total: 2.83s	remaining: 5.2s
176:	learn: 8.4191008	total: 2.84s	remaining: 5.18s
177:	learn: 8.4072269	total: 2.85s	remaining: 5.16s
178:	learn: 8.3959368	total: 2.86s	remaining: 5.14s
179:	learn: 8.3875590	total: 2.88s	remaining: 5.13s
180:	learn: 8.3820446	total: 2.9s	remaining: 5.11s
181:	learn: 8.3710061	total: 2.91s	remaining: 5.08s
182:	learn: 8.3665293	total: 2.93s	remaining: 5.08s
183:	learn: 8.3641785	total: 2.95s	remaining: 5.06s
184:	learn: 8.3560411	total: 2.96s	remaining: 5.04s
185:	learn: 8.3437426	total: 2.98s	remaining: 5.03s
186:	learn: 8.3336683	total: 2.99s	remaining: 5.01s
187:	learn: 8.3235286	total: 3.03s	remaining: 5.03s
188:	learn: 8.3167347	total: 3.04s	remaining: 5.01s
189:	learn: 8.3

331:	learn: 7.3603824	total: 5.07s	remaining: 2.57s
332:	learn: 7.3532384	total: 5.09s	remaining: 2.55s
333:	learn: 7.3459218	total: 5.11s	remaining: 2.54s
334:	learn: 7.3386203	total: 5.12s	remaining: 2.52s
335:	learn: 7.3292194	total: 5.13s	remaining: 2.5s
336:	learn: 7.3264173	total: 5.15s	remaining: 2.49s
337:	learn: 7.3182831	total: 5.16s	remaining: 2.47s
338:	learn: 7.3121836	total: 5.17s	remaining: 2.46s
339:	learn: 7.3087869	total: 5.19s	remaining: 2.44s
340:	learn: 7.3022902	total: 5.2s	remaining: 2.43s
341:	learn: 7.2975942	total: 5.21s	remaining: 2.41s
342:	learn: 7.2880070	total: 5.23s	remaining: 2.39s
343:	learn: 7.2814481	total: 5.25s	remaining: 2.38s
344:	learn: 7.2650330	total: 5.25s	remaining: 2.36s
345:	learn: 7.2613760	total: 5.26s	remaining: 2.34s
346:	learn: 7.2547378	total: 5.28s	remaining: 2.33s
347:	learn: 7.2501245	total: 5.3s	remaining: 2.31s
348:	learn: 7.2344646	total: 5.31s	remaining: 2.3s
349:	learn: 7.2290530	total: 5.32s	remaining: 2.28s
350:	learn: 7.22

StackingRegressor(cv=KFold(n_splits=5, random_state=1, shuffle=True),
                  estimators=[('lasso', Lasso(alpha=0.05)),
                              ('catboost',
                               <catboost.core.CatBoostRegressor object at 0x7f899a4c0b20>)],
                  final_estimator=LinearRegression())

In [20]:
pred = pd.DataFrame()
pred['id'] = test['id']
pred['y'] = en.predict(X_test_pca)
pred = pred.set_index(['id'])
pred.to_csv('Witarsa_Ashley_regression.csv') # RMSE: 9.637 